## Plotting lines with capacity ##

In [14]:
import pandas as pd
import sys
import numpy as np
import os
import logging
import re
from pathlib import Path
from functools import wraps
import io
import geopandas as gpd
import folium
from shapely.geometry import Polygon

sys.path.insert(0, './postprocessing')
from utils import *
from plots import *

# Verify the file exists before reading
output_file = "../output/simulations_run_20251216_150832/baseline/output_csv/pDispatch.csv"
if os.path.exists(output_file):
	pDispatch = pd.read_csv(output_file)
else:
	print(f"Warning: File not found: {output_file}")
	print(f"Available files in current directory: {os.listdir('.')}")
	# Uncomment and adjust the path below once you confirm the correct location
	# pDispatch_df = pd.read_csv("path/to/correct/pDispatch.csv")

In [26]:
exchange_components = filter_dataframe(pDispatch, {'uni': ["Imports", "Exports"]})


z     y      uni       value
q  d  t                                           
Q1 d1 t1   South_Africa  2025  Imports  271.132542
      t2   South_Africa  2025  Imports  284.862346
      t3   South_Africa  2025  Imports   69.854085
      t4   South_Africa  2025  Imports   49.692150
      t5   South_Africa  2025  Imports  298.592150
...                 ...   ...      ...         ...
Q4 d4 t16         Mozal  2035  Exports  -58.172169
      t21         Mozal  2035  Exports  -43.292450
      t22         Mozal  2035  Exports  -43.292450
      t23         Mozal  2035  Exports  -43.292450
      t24         Mozal  2035  Exports  -38.332543

[68918 rows x 4 columns]

In [31]:
df_plot = exchange_components.copy()
df_plot = df_plot.set_index(['q', 'd','t'])
print(df_plot.head())

                     z     y      uni       value
q  d  t                                          
Q1 d1 t1  South_Africa  2025  Imports  271.132542
      t2  South_Africa  2025  Imports  284.862346
      t3  South_Africa  2025  Imports   69.854085
      t4  South_Africa  2025  Imports   49.692150
      t5  South_Africa  2025  Imports  298.592150


In [ ]:
df_plot = exchange_components.copy()

df_plot = df_plot.set_index(['q', 'd','t'])
df_plot = df_plot.drop(columns=['uni'])
df_plot_init=df_plot[df_plot['y']==2025].drop(columns=['y'])
#df_plot_init.unstack().plot(kind = 'bar', stacked  =True)
#print(df_plot_init.head())

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
df_plot.plot.bar(stacked  =True)

In [ ]:
# Plot df_plot_init as stacked bar with countries (z) stacked
df_to_plot = df_plot_init.reset_index()

# Create combined x-axis label (season|day|time)
df_to_plot['qdt'] = df_to_plot['q'].astype(str) + '|' + df_to_plot['d'].astype(str) + '|' + df_to_plot['t'].astype(str)

# Call make_stacked_barplot: stacked by zone (z), x-axis organized by qdt
make_stacked_barplot(
    df_to_plot,
    filename=None,  # Set to a path (e.g., 'stacked_bar_2025.png') to save
    dict_colors=None,  # Optional: pass a dict mapping zones to colors
    column_subplot=None,  # No subplots for single year
    column_stacked='z',  # Stack by country
    column_xaxis='qdt',  # X-axis groups by season|day|time
    column_value='value',  # Values to stack
    annotate=False,  # Set to True to add value labels
    figsize=(16, 6),
    show_legend=True
)

KeyError: 'qdt'

In [12]:
_GEOJSON_HEADER = "Geojson,EPM,region,country,division"


def _read_geojson_mapping(path):
    """
    Read a GeoJSON-to-EPM mapping CSV while normalizing repeated header rows.

    Some exported CSVs append the header again without a newline, causing pandas
    to fail when parsing. This helper inserts the missing newline and removes
    duplicate header rows so that `pd.read_csv` can succeed.
    """
    with open(path, encoding='utf-8-sig') as fp:
        raw_text = fp.read()

    if _GEOJSON_HEADER not in raw_text:
        return pd.read_csv(path)

    pattern = r'(?<=.)(?<![\r\n])' + re.escape(_GEOJSON_HEADER)
    normalized_text = re.sub(pattern, '\n' + _GEOJSON_HEADER, raw_text)

    clean_lines = []
    header_seen = False
    for line in normalized_text.splitlines():
        if line.strip() == _GEOJSON_HEADER:
            if header_seen:
                continue
            header_seen = True
        clean_lines.append(line)

    clean_text = "\n".join(clean_lines)
    if not clean_text.endswith("\n"):
        clean_text += "\n"

    return pd.read_csv(io.StringIO(clean_text))

def create_zonemap(zone_map, map_geojson_to_epm):
    """
    Convert zone map to the correct coordinate reference system (CRS) and extract centroids.

    This function ensures that the provided `zone_map` is in EPSG:4326 (latitude/longitude),
    extracts the centroid coordinates of each zone, and maps them to the EPM zone names.

    Parameters
    ----------
    zone_map : gpd.GeoDataFrame
        A GeoDataFrame containing zone geometries and attributes.
    map_geojson_to_epm : dict
        Dictionary mapping GeoJSON zone names to EPM zone names.

    Returns
    -------
    tuple
        - zone_map (gpd.GeoDataFrame): The zone map converted to EPSG:4326.
        - centers (dict): Dictionary mapping EPM zone names to their centroid coordinates [longitude, latitude].
    """
    if zone_map.crs is not None and zone_map.crs.to_epsg() != 4326:
        zone_map = zone_map.to_crs(epsg=4326)  # Convert to EPSG:4326 for folium

    # Get the coordinates of the centers of the zones
    centers = {
        row['ADMIN']: [row.geometry.centroid.x, row.geometry.centroid.y]
        for _, row in zone_map.iterrows()
    }

    centers = {map_geojson_to_epm[c]: v for c, v in centers.items() if c in map_geojson_to_epm}

    return zone_map, centers


def get_json_data(epm_results=None, selected_zones=None, dict_specs=None, geojson_to_epm=None, geo_add=None,
                  zone_map=None):
    """
    Extract and process zone map data, handling divisions for sub-national regions.

    This function retrieves the zone map, identifies zones that need to be divided
    (e.g., North-South or East-West split), applies the `divide` function, and
    returns a processed GeoDataFrame ready for visualization.

    Parameters
    ----------
    epm_results : dict
        Dictionary containing EPM results, including transmission capacity data.
    dict_specs : dict
        Dictionary with mapping specifications, including:
        - `geojson_to_epm`: Mapping from GeoJSON names to EPM zone names.
        - `map_zones`: GeoDataFrame of all countries.

    Returns
    -------
    tuple
        - zone_map (gpd.GeoDataFrame): Processed zone map including divided regions.
        - geojson_to_epm (dict): Updated mapping of GeoJSON names to EPM zones.
    """
    assert ((dict_specs is not None) or (geojson_to_epm is not None)), "Mapping zone names from geojson to EPM must be provided either under dict_specs or under geojson_to_epm"

    if dict_specs is None:
        if 'postprocessing' in os.getcwd():
            dict_specs = read_plot_specs(folder='')
        else:
            dict_specs = read_plot_specs(folder='postprocessing')
    if geojson_to_epm is None:
        geojson_to_epm = dict_specs['geojson_to_epm']
    else:
        if not os.path.exists(geojson_to_epm):
            raise FileNotFoundError(f"GeoJSON to EPM mapping file not found: {os.path.abspath(geojson_to_epm)}")
        geojson_to_epm = _read_geojson_mapping(geojson_to_epm)
    epm_to_geojson = {v: k for k, v in
                      geojson_to_epm.set_index('Geojson')['EPM'].to_dict().items()}  # Reverse dictionary
    geojson_to_divide = geojson_to_epm.loc[geojson_to_epm.region.notna()]
    geojson_complete = geojson_to_epm.loc[~geojson_to_epm.region.notna()]
    if selected_zones is None:
        selected_zones_epm = geojson_to_epm['EPM'].unique()
    else:
        selected_zones_epm = selected_zones
    selected_zones_to_divide = [e for e in selected_zones_epm if e in geojson_to_divide['EPM'].values]
    selected_countries_geojson = [
        epm_to_geojson[key] for key in selected_zones_epm if
        ((key not in selected_zones_to_divide) and (key in epm_to_geojson))
    ]

    if zone_map is None:
        zone_map = dict_specs['map_zones']  # getting json data on all countries
    else:
        zone_map = gpd.read_file(zone_map)

    zone_map = zone_map[zone_map['ADMIN'].isin(selected_countries_geojson)]

    if geo_add is not None:
        zone_map_add = gpd.read_file(geo_add)
        zone_map = pd.concat([zone_map, zone_map_add])

    divided_parts = []
    for (country, division), subset in geojson_to_divide.groupby(['country', 'division']):
        # Apply division function
        divided_parts.append(divide(dict_specs['map_zones'], country, division))

    if divided_parts:
        zone_map_divide = pd.concat(divided_parts)

        zone_map_divide = \
        geojson_to_divide.rename(columns={'country': 'ADMIN'}).merge(zone_map_divide, on=['region', 'ADMIN'])[
            ['Geojson', 'ISO_A3', 'ISO_A2', 'geometry']]
        zone_map_divide = zone_map_divide.rename(columns={'Geojson': 'ADMIN'})
        # Convert zone_map_divide back to a GeoDataFrame
        zone_map_divide = gpd.GeoDataFrame(zone_map_divide, geometry='geometry', crs=zone_map.crs)

        # Ensure final zone_map is in EPSG:4326
        zone_map = pd.concat([zone_map, zone_map_divide]).to_crs(epsg=4326)
    geojson_to_epm = geojson_to_epm.set_index('Geojson')['EPM'].to_dict()  # get only relevant info
    return zone_map, geojson_to_epm


def get_value(df, zone, year, scenario, attribute, column_to_select='attribute'):
    """Safely retrieves an energy value for a given zone, year, scenario, and attribute."""
    value = df.loc[
        (df['zone'] == zone) & (df['year'] == year) & (df['scenario'] == scenario) & (df[column_to_select] == attribute),
        'value'
    ]
    return value.values[0] if not value.empty else 0


def divide(geodf, country, division):
    """
    Divide a country's geometry into two subzones using North-South (NS) or East-West (EW) division.

    This function overlays the country geometry with a dividing polygon and extracts
    the two subregions.

    Parameters
    ----------
    geodf : gpd.GeoDataFrame
        GeoDataFrame containing geometries of all countries.
    country : str
        Name of the country to divide.
    division : str
        Type of division:
        - 'NS' (North-South) splits along the latitude midpoint.
        - 'EW' (East-West) splits along the longitude midpoint.
        - 'NSE' (North-South-East) splits into three quadrants.

    Returns
    -------
    gpd.GeoDataFrame
        GeoDataFrame containing the divided subregions with the correct CRS.
    """
    # Get the country geometry
    crs = geodf.crs
    country_geometry = geodf.loc[geodf['ADMIN'] == country, 'geometry'].values[0]

    # Get bounds
    minx, miny, maxx, maxy = country_geometry.bounds

    if division == 'NS':
        median_latitude = (miny + maxy) / 2
        south_polygon = Polygon([(minx, miny), (minx, median_latitude), (maxx, median_latitude), (maxx, miny)])
        north_polygon = Polygon([(minx, median_latitude), (minx, maxy), (maxx, maxy), (maxx, median_latitude)])

        # Convert to GeoDataFrame with the correct CRS
        south_gdf = gpd.GeoDataFrame(geometry=[south_polygon], crs=crs)
        north_gdf = gpd.GeoDataFrame(geometry=[north_polygon], crs=crs)

        south_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], south_gdf, how='intersection')
        north_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], north_gdf, how='intersection')
        south_part = south_part.to_crs(crs)
        north_part = north_part.to_crs(crs)
        south_part['region'] = 'south'
        north_part['region'] = 'north'

        return pd.concat([south_part, north_part])
    
    elif division == 'EW':
        median_longitude = (maxx-minx) / 2
        median_limit = minx + median_longitude
        
        west_polygon = Polygon([(minx, miny), (minx, maxy), (median_limit, maxy), (median_limit, miny)])
        east_polygon = Polygon([(median_limit, miny), (median_limit, maxy), (maxx, maxy), (maxx, miny)])

        # Convert to GeoDataFrame with the correct CRS
        west_gdf = gpd.GeoDataFrame(geometry=[west_polygon], crs=crs)
        east_gdf = gpd.GeoDataFrame(geometry=[east_polygon], crs=crs)

        west_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country],west_gdf, how='intersection')
        east_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], east_gdf, how='intersection')
        west_part['region'] = 'west'
        east_part['region'] = 'east'
        
        return pd.concat([west_part, east_part])
        
    elif division == 'NCS':
         
        third_latitude = (maxy - miny) / 3
        south_limit = miny + third_latitude
        north_limit = maxy - third_latitude

        south_polygon = Polygon([(minx, miny), (minx, south_limit), (maxx, south_limit), (maxx, miny)])
        center_polygon = Polygon([(minx, south_limit), (minx, north_limit), (maxx, north_limit), (maxx, south_limit)])
        north_polygon = Polygon([(minx, north_limit), (minx, maxy), (maxx, maxy), (maxx, north_limit)])

        south_gdf = gpd.GeoDataFrame(geometry=[south_polygon], crs=crs)
        center_gdf = gpd.GeoDataFrame(geometry=[center_polygon], crs=crs)
        north_gdf = gpd.GeoDataFrame(geometry=[north_polygon], crs=crs)

        south_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], south_gdf, how='intersection')
        center_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], center_gdf, how='intersection')
        north_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], north_gdf, how='intersection')

        south_part['region'] = 'south'
        center_part['region'] = 'center'
        north_part['region'] = 'north'

        return pd.concat([north_part, center_part, south_part])
         
        
    elif division == 'NSE':
        median_latitude = (miny + maxy) / 2
        median_longitude = (minx + maxx) / 2
        north_polygon = Polygon([(minx, median_latitude), (minx, maxy), (median_longitude, maxy), (median_longitude, median_latitude)])
        south_polygon = Polygon([(minx, miny), (minx, median_latitude), (median_longitude, median_latitude), (median_longitude, miny)])
        east_polygon = Polygon([(median_longitude, miny), (median_longitude, median_latitude), (maxx, median_latitude), (maxx, miny)])
        west_polygon = Polygon([(minx, median_latitude), (minx, maxy), (median_longitude, maxy), (median_longitude, median_latitude)])
        # Convert to GeoDataFrame with the correct CRS
        north_gdf = gpd.GeoDataFrame(geometry=[north_polygon], crs=crs)
        south_gdf = gpd.GeoDataFrame(geometry=[south_polygon], crs= crs)
        east_gdf = gpd.GeoDataFrame(geometry=[east_polygon], crs= crs)
        west_gdf = gpd.GeoDataFrame(geometry=[west_polygon], crs= crs)
        north_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], north_gdf, how='intersection')
        south_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], south_gdf, how='intersection')
        east_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], east_gdf, how='intersection')
        west_part = gpd.overlay(geodf.loc[geodf['ADMIN'] == country], west_gdf, how='intersection')
        north_part['region'] = 'north'
        south_part['region'] = 'south'
        east_part['region'] = 'east'
        west_part['region'] = 'west'

        return pd.concat([west_part, east_part, north_part, south_part])

    else:
        raise ValueError("Invalid division type. Use 'NS' (North-South) , 'NSE' (North-South-East') or 'EW' (East-West).")

In [13]:
geojson_to_epm_path = "C:\\Users\\wb590499\\Documents\\Projects\\EPM\\epm\\postprocessing\\static\\geojson_to_epm_bis.csv"

In [14]:
zone_map= "C:\\Users\\wb590499\\Documents\\Projects\\EPM\\epm\\postprocessing\\static\\zones.geojson"
dict_specs = {
        'map_zones': gpd.read_file(zone_map),
    }

In [15]:
zone_map, geojson_to_epm_dict = get_json_data(epm_results = None, dict_specs = dict_specs, geojson_to_epm = geojson_to_epm_path,
                                                  zone_map=zone_map)

In [16]:
zone_map, centers = create_zonemap(zone_map, map_geojson_to_epm=geojson_to_epm_path)

In [17]:
bounds = zone_map.total_bounds  # [minx, miny, maxx, maxy]
region_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]  # Center latitude, longitude
energy_map = folium.Map(location=region_center, zoom_start=6, tiles='CartoDB positron')

# Add country zones
folium.GeoJson(zone_map, style_function=lambda feature: {
        'fillColor': '#ffffff', 'color': '#000000', 'weight': 1, 'fillOpacity': 0.3
}).add_to(energy_map)
energy_map.save("footprint.html")